# Parsing and Inspecting PDDL problems

Let's use the built-in PDDL parser to inspect some problem encoded in PDDL:

In [16]:
from tarski.io import PDDLReader
reader = PDDLReader(raise_on_error=True)
reader.parse_domain('./benchmarks/blocksworld.pddl')
problem = reader.parse_instance('./benchmarks/probBLOCKS-4-2.pddl')
lang = problem.language

Notice how the parsing of a standard instance of the Blocks world results in
a _problem_ object and, within that problem, a _language_ object. There
is a clear distinction in Tarski between the language used to define a planning
problem, and the problem itself. Tarski sticks as close as possible to the
standard definition of
[many-sorted first-order languages](https://en.wikipedia.org/wiki/First-order_logic#Many-sorted_logic).
Hence, languages have a vocabulary made up of predicate and function names,
each with their arity and sort, and allow the definition of terms and formulas
in the usual manner. Let us inspect the language of the problem we just parsed: 

In [17]:
print(lang.sorts)
print(lang.predicates)
print(lang.functions)

[Sort(object)]
[=/2, !=/2, on/2, ontable/1, clear/1, handempty/0, holding/1]
[]


Turns out that our blocks encoding has one single sort `object` (the default
sort in PDDL when no sort is declared), no functions, and a few predicates.
Besides the predicates defined in the PDDL file, Tarski assumes (unless explicitly
disallowed) the existence of a built-in equality predicate and its negation.
A string `clear/1` denotes a predicate named `clear` with arity 1. 

Constants are usually considered as nullary functions in the literature, but in
Tarski they are stored separately:

In [18]:
lang.constants()

[b (object), d (object), c (object), a (object)]

Our blocks encoding this has four constants of type object, which we know
represent the four different blocks in the problem. 

Languages also provide means to directly retrieve any sort, constant, function or predicate
when their name is known: 

In [19]:
print(lang.get('on'))
print(lang.get('clear'))
print(lang.get('a', 'b', 'c'))

on/2
clear/1
(a (object), b (object), c (object))


Notice how in the last statement we retrieve three different elements (in this case, constants) in one single call. 

We can also easily inspect all constants of a certain sort:

In [20]:
list(lang.get('object').domain())

[b (object), c (object), d (object), a (object)]

We can of course inspect not only the language, but also the problem itself.

In [21]:
problem

Problem(name="BLOCKS-4-2", domain="BLOCKS")

A PDDL planning problem is made up of some initial state, some goal formula, 
and a number of actions. The initial state is essentially an encoding of 
a first-order interpretation over the language of the problem:

In [22]:
problem.init

Model(num_predicates="4", num_functions="0")

We can also get an extensional description of our initial state:

In [23]:
print(problem.init.as_atoms())

[clear(d), clear(a), clear(c), ontable(d), ontable(b), ontable(a), on(c,b), handempty()]


Since the initial state is just another Tarski model, we can perform with it all operations that
we have seen on previous tutorials, e.g. inspecting the value of some atom on it:

In [24]:
clear, b = lang.get('clear', 'b')
problem.init[clear(b)]


False

In contrast with the initial state, the goal can be any arbitrary first-order formula.
Unsurprisingly, it turns out that the goal is not satisfied in the initial state;

In [25]:
print(problem.goal)
print(problem.init[problem.goal])

(on(a,b) and on(b,c) and on(c,d))
False


Our blocks encoding has four actions:


In [26]:
print(list(problem.actions))


['pick-up', 'put-down', 'stack', 'unstack']


We can of course retrieve and inspect each action individually:

In [27]:
stack = problem.get_action('stack')
stack.parameters

?x,?y

In [28]:
stack.precondition

(holding(?x) and clear(?y))

In [29]:
stack.effects


[(T -> DEL(holding(?x))),
 (T -> DEL(clear(?y))),
 (T -> ADD(clear(?x))),
 (T -> ADD(handempty())),
 (T -> ADD(on(?x,?y)))]